In [ ]:
import tensorflow as tf
import numpy as np

2023-10-13 13:39:46.675566: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-13 13:39:46.677419: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 13:39:46.715691: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 13:39:46.716351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 13:39:47.473413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# Set Model & Hyperparameters

In [ ]:
MODEL_NAME = 1693815807

PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000,
}

LABELS = ['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

# Instantiate Pre-processing

In [ ]:
from preprocessing import AudioReader
from preprocessing import MelSpectrogram


audio_reader = AudioReader(tf.int16, PREPROCESSING_ARGS['sampling_rate'])
mel_spec_processor = MelSpectrogram(**PREPROCESSING_ARGS)

# Load the TFLite Model

In [ ]:
interpreter = tf.lite.Interpreter(model_path=f'tflite_models/{MODEL_NAME}.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output name:", output_details[0]['name'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_input_1:0
Input shape: [ 1 49 40  1]
Output name: StatefulPartitionedCall:0
Output shape: [1 8]
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Test the TFLite Model

In [ ]:
from glob import glob
from time import time

filenames = glob('msc-test/*')

preprocessing_latencies = []
model_latencies = []
tot_latencies = []
accuracy = 0.0

for filename in filenames:
    audio, true_label = audio_reader.get_audio_and_label(filename)   
    true_label = true_label.numpy().decode()
    
    start_preprocess = time()
    log_mel_spectrogram = mel_spec_processor.get_mel_spec(audio)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, 0)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)
    end_preprocess = time()

    interpreter.set_tensor(input_details[0]['index'], log_mel_spectrogram)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    end_inference = time()

    top_index = np.argmax(output[0])
    predicted_label = LABELS[top_index]

    accuracy += true_label == predicted_label
    preprocessing_latencies.append(end_preprocess - start_preprocess)
    model_latencies.append(end_inference - end_preprocess)
    tot_latencies.append(end_inference - start_preprocess) 

2023-10-13 13:39:48.686572: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 AVX512F FMA
2023-10-13 13:39:48.688256: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [ ]:
accuracy /= len(filenames)
median_preprocessing_latency = np.median(preprocessing_latencies)
median_model_latency = np.median(model_latencies)
median_total_latency = np.median(tot_latencies)

import os

model_size = os.path.getsize(f'tflite_models/{MODEL_NAME}.tflite')

In [ ]:
print(tot_latencies)

[0.05570268630981445, 0.012085199356079102, 0.011726617813110352, 0.011746406555175781, 0.01139521598815918, 0.011517763137817383, 0.011371135711669922, 0.01179194450378418, 0.011465311050415039, 0.012388467788696289, 0.011367559432983398, 0.011359214782714844, 0.01162862777709961, 0.01169443130493164, 0.01161336898803711, 0.011521339416503906, 0.011467218399047852, 0.011735916137695312, 0.012641668319702148, 0.0169830322265625, 0.012213706970214844, 0.013053178787231445, 0.011676788330078125, 0.011618852615356445, 0.011747360229492188, 0.011395454406738281, 0.011948347091674805, 0.012906074523925781, 0.011964559555053711, 0.012554407119750977, 0.01311349868774414, 0.011802911758422852, 0.011409759521484375, 0.011415243148803711, 0.011368036270141602, 0.011281490325927734, 0.01160120964050293, 0.011584043502807617, 0.011463642120361328, 0.011737823486328125, 0.01145482063293457, 0.012134313583374023, 0.01170039176940918, 0.012290477752685547, 0.011894464492797852, 0.013290166854858398,

# Report the Test Results

In [ ]:
print(f'Accuracy: {100 * accuracy:.3f}%')
print(f'Model size: {model_size / 2 ** 10:.1f}KB')
print(f'Preprocessing Latency: {1000 * median_preprocessing_latency:.1f}ms')
print(f'Model Latency: {1000 * median_model_latency:.1f}ms')
print(f'Total Latency: {1000 * median_total_latency:.1f}ms')

Accuracy: 90.250%
Model size: 1165.1KB
Preprocessing Latency: 8.2ms
Model Latency: 3.9ms
Total Latency: 12.2ms


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f1fd91f-a434-4542-983d-3ce5ae14ac33' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>